In [1]:
# 主要步驟
#     讀取資料
#         讀檔案
#         取出常用變數(若有更改常用變數時要記得更新
#     清洗資料
#         刪除價格值為0的奇怪資料
#         刪除不會用到的欄位
#         刪除不會用到的列
#         找出資料誤值或空值
#         驗算資料
#     轉換資料
#         修正日期格式
#         以代號取代原資料(市場>>代號、花名重編、分類)
#         將各種花合併得到總交易量、平均價格
#         新增已周圍單位的存檔
#     存取資料

In [2]:
# seting 讀取檔案
import os, glob
import numpy as np
import pandas as pd
from datetime import date, time, datetime
from datetime import timedelta

fn_list = glob.glob("../result/merge_flower_price/*台北.csv")
flowers =[]
for fn in fn_list:
    flowers.append(fn.split("\\")[-1].split("_")[0])
# flowers
datasets =[]
for f in flowers:
    key_word = "../result/merge_flower_price/" + f +"*.csv"
    datasets.append(glob.glob(key_word))
datasets
# print(fn_list)
# fn = fn_list[4]


[['../result/merge_flower_price\\Anthurium_台中.csv',
  '../result/merge_flower_price\\Anthurium_台北.csv',
  '../result/merge_flower_price\\Anthurium_台南.csv',
  '../result/merge_flower_price\\Anthurium_彰化.csv',
  '../result/merge_flower_price\\Anthurium_高雄.csv'],
 ['../result/merge_flower_price\\Chrysanthemum_台中.csv',
  '../result/merge_flower_price\\Chrysanthemum_台北.csv',
  '../result/merge_flower_price\\Chrysanthemum_台南.csv',
  '../result/merge_flower_price\\Chrysanthemum_彰化.csv',
  '../result/merge_flower_price\\Chrysanthemum_高雄.csv'],
 ['../result/merge_flower_price\\Eustoma_台中.csv',
  '../result/merge_flower_price\\Eustoma_台北.csv',
  '../result/merge_flower_price\\Eustoma_台南.csv',
  '../result/merge_flower_price\\Eustoma_彰化.csv',
  '../result/merge_flower_price\\Eustoma_高雄.csv'],
 ['../result/merge_flower_price\\OrientalLily_台中.csv',
  '../result/merge_flower_price\\OrientalLily_台北.csv',
  '../result/merge_flower_price\\OrientalLily_台南.csv',
  '../result/merge_flower_price\\OrientalL

In [3]:
def merge_all_market(fn_list):
    df_list = []
    for fn in fn_list:
        df_list.append(pd.read_csv(fn, 
                                   names=["date", "market", "price", "count", "year", "week"],
                                   encoding="utf-8"))
    len(df_list)
    df_new = df_list[0].append([df_list[1],df_list[2],df_list[3],df_list[4]])
    df_new = df_new.reset_index()
    df_new = df_new.drop(columns="index")
    df_m_by_date = df_new.copy()
    df_m_by_date["flower"] = flower
    arr = np.array(df_new)
    freq = df_new["date"].value_counts()
    all_date = df_new["date"].drop_duplicates()
#     null_data = df.iloc[df["date"].index.isnull()]
    merge_data = []
    c = ["date", "avg_price", "total_count", "year", "week"]
    for d in all_date:
    #     day_data =[]
        sub_data = arr[arr[:,0]==d,:]
        d_count = sum(sub_data[:,3])
        d_avg = round(sum(sub_data[:,3]*sub_data[:,2])/d_count,3)
#         print(sub_data)

        day_data =[sub_data[0,0], d_avg, d_count, sub_data[0,4], sub_data[0,5]]
        merge_data.append(day_data)
    df_m = pd.DataFrame(merge_data, columns=c)
    df_m["flower"] = flower
    return df_m, df_m_by_date

In [4]:
for i in range(len(datasets)):
    dataset = datasets[i]
#     print(dataset)
    flower = flowers[i]
    df_result, df_by_day = merge_all_market(dataset)
    fn_save = "../result/merge_flower_allmarket_price/merge_" + flower + ".csv"
    df_result.to_csv(fn_save, index=False, encoding="utf-8")
    fn_save = "../result/merge_flower_allmarket_price/mongo_up_" + flower + ".csv"
    df_result.to_csv(fn_save, index=False, encoding="utf-8", header=False)
    fn_save = "../result/merge_flower_price_by_day/" + flower + ".csv"
    df_by_day.to_csv(fn_save, index=False, encoding="utf-8")
    print("data saved", flower)


data saved Anthurium
data saved Chrysanthemum
data saved Eustoma
data saved OrientalLily
data saved Rose


In [9]:
df_result.tail(4)

,date,avg_price,total_count,year,week,flower
1834,2018-09-30,57.67,119,2018,39,Rose
1835,2018-10-07,105.06,159,2018,40,Rose
1836,2018-10-14,132.08,168,2018,41,Rose
1837,2018-10-21,87.33,503,2018,42,Rose


In [10]:
df_by_day.tail(5)

,date,market,price,count,year,week,flower
8708,2018-05-30,高雄,65.49,1144,2018,22,Rose
8709,2018-05-31,高雄,100.44,1286,2018,22,Rose
8710,2018-06-01,高雄,77.64,1162,2018,22,Rose
8711,2018-06-02,高雄,55.25,879,2018,22,Rose
8712,2018-06-04,高雄,73.18,1324,2018,23,Rose


In [7]:
import matplotlib.pyplot as plt
plt.plot(df.index.to_list(), df["avg_price"])
plt.show()

NameError: name 'df' is not defined